#  Installing and importing packages

In [1]:
pip install betterreads

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from betterreads import client
from goodreads_data_gather import helper

Gathering data... 
[   20.    21.    22. ...  9998.  9999. 10000.]
getting 10 out of missing 9981 descriptions
20.0,Mockingjay (The Hunger Games, #3)
saving the results to the file....
21.0,Harry Potter and the Order of the Phoenix (Harry Potter, #5)
22.0,The Lovely Bones
23.0,Harry Potter and the Chamber of Secrets (Harry Potter, #2)
24.0,Harry Potter and the Goblet of Fire (Harry Potter, #4)
25.0,Harry Potter and the Deathly Hallows (Harry Potter, #7)
26.0,The Da Vinci Code (Robert Langdon, #2)
27.0,Harry Potter and the Half-Blood Prince (Harry Potter, #6)
28.0,Lord of the Flies
29.0,Romeo and Juliet


# Data Gathering

In [3]:
h = helper()

Gathering data... 


In [4]:
### Note : Only run this very first run so we can get the orginal books set

In [5]:
books = pd.read_csv('../data/books.csv')
print( f"shape of the books dataframe is {books.shape}")
#We are going to add. 3 more columns and populate them using the GoodReads client
books['description'] =np.nan
books['num_pages'] =0
books['e_book'] =False
books.to_csv("../data/books_desc.csv", index=False)
books.columns

shape of the books dataframe is (10000, 23)


Index(['book_id', 'goodreads_book_id', 'best_book_id', 'work_id',
       'books_count', 'isbn', 'isbn13', 'authors', 'original_publication_year',
       'original_title', 'title', 'language_code', 'average_rating',
       'ratings_count', 'work_ratings_count', 'work_text_reviews_count',
       'ratings_1', 'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5',
       'image_url', 'small_image_url', 'description', 'num_pages', 'e_book'],
      dtype='object')

In [6]:
books =pd.read_csv("../data/books_desc.csv")

ratings = pd.read_csv("../data/ratings.csv")
print( f"shape of the ratings dataframe is {ratings.shape}")

book_tags = pd.read_csv("../data/book_tags.csv")
print( f"shape of the books_tags dataframe is {book_tags.shape}")

tags = pd.read_csv("../data/tags.csv")
print( f"shape of the tags dataframe is {tags.shape}")

### Combining booktag ids with the actual names
pd.merge(left=book_tags,right =tags, how ="left", on ='tag_id')
book_tags.to_csv("../data/book_tags_combined",index =False)
book_tags = pd.read_csv("../data/book_tags_combined.csv")
books.head(1)

shape of the ratings dataframe is (5976479, 3)
shape of the books_tags dataframe is (999912, 3)
shape of the tags dataframe is (34252, 2)


,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,description,num_pages,e_book
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,NaN,0,False


In [7]:
tags.head(1)

,tag_id,tag_name
0,0,-


In [8]:
book_tags.head(1)

,goodreads_book_id,tag_id,count
0,1,30574,167697


In [9]:
ratings.head(1)

,user_id,book_id,rating
0,1,258,5


In [10]:
### We are going to populate description , num_of_pages and is_ebook from goodreads Api

In [11]:
books =h.fill_missing_columns("../data/books_desc.csv",10)

[    1     2     3 ...  9998  9999 10000]
getting 10 out of missing 10000 descriptions
1,The Hunger Games (The Hunger Games, #1)
2,Harry Potter and the Sorcerer's Stone (Harry Potter, #1)
3,Twilight (Twilight, #1)
4,To Kill a Mockingbird
5,The Great Gatsby
6,The Fault in Our Stars
7,The Hobbit, or There and Back Again
8,The Catcher in the Rye
9,Angels & Demons (Robert Langdon, #1)
10,Pride and Prejudice
saving the results to the file....


585 books that are missing orig title, replace them with title
book = gc.book(3)
book.original_title

### Collecting  recent books  based on Denver DSI classmates reading preferences

In [12]:
collect_books = pd.DataFrame()
topics =['python','of Thrones','Patrick Rothfuss','A Song of Ice and Fire','Inspector Rebus',
      'Data Science', 'mechine learning','Data Visualization','Robert T. Kiyosaki','Brené Brown',
      'weapons of math destruction','Girl, Wash Your Face','recommender systems']
for topic in topics :
    df2 =h.get_books(topic)
    books = pd.concat([books,df2],sort=True)
books.reset_index(inplace=True, drop=True)  
books.to_csv("../data/books_desc.csv", index=False)

In [13]:
books.columns

Index(['authors', 'average_rating', 'best_book_id', 'book_id', 'books_count',
       'description', 'e_book', 'goodreads_book_id', 'image_url', 'is_ebook',
       'isbn', 'isbn13', 'language_code', 'num_pages',
       'original_publication_year', 'original_title', 'ratings_1', 'ratings_2',
       'ratings_3', 'ratings_4', 'ratings_5', 'ratings_count',
       'small_image_url', 'title', 'work_id', 'work_ratings_count',
       'work_text_reviews_count'],
      dtype='object')

### Data Cleaning

In [14]:
###### How many of the  columns are missing data ?
books.isna().sum()

authors                         0
average_rating                  0
best_book_id                  137
book_id                       137
books_count                   137
description                  9990
e_book                        137
goodreads_book_id               0
image_url                     137
is_ebook                     9990
isbn                          837
isbn13                        722
language_code                1137
num_pages                       0
original_publication_year     158
original_title                722
ratings_1                     137
ratings_2                     137
ratings_3                     137
ratings_4                     137
ratings_5                     137
ratings_count                   0
small_image_url               137
title                           0
work_id                       137
work_ratings_count            137
work_text_reviews_count       137
dtype: int64

In [15]:
print(f"{len(books)}, {len(books[books['original_title'].isna()])},{len(books[books['original_title'] != books ['title']])}")
# Title seems to be including original title in all these cases, we are going to drop original_title
books[books['original_title'] != books ['title']][['goodreads_book_id','title','original_title']]

10137, 722,6152


,goodreads_book_id,title,original_title
0,2767052,"The Hunger Games (The Hunger Games, #1)",The Hunger Games
1,3,Harry Potter and the Sorcerer's Stone (Harry P...,Harry Potter and the Philosopher's Stone
2,41865,"Twilight (Twilight, #1)",Twilight
6,5907,The Hobbit,The Hobbit or There and Back Again
8,960,"Angels & Demons (Robert Langdon, #1)",Angels & Demons
...,...,...,...
10132,23876976,Recommender Systems,NaN
10133,41310766,Recommender System,NaN
10134,27136807,Social Network-Based Recommender Systems,NaN
10135,13379246,Recommender Systems for Social Tagging Systems,NaN


In [16]:
#Drop columns that are not relevant for the recommender
# books = books.drop(['best_book_id','work_id','isbn13','isbn','original_title',
#                     'books_count','work_ratings_count','work_text_reviews_count',
#                     'image_url','small_image_url'], axis =1)

In [17]:
###### If the book description was empty ,  adding a string so that  Vecotrizer can  work with the data
books['description'] = books['description'].fillna('-')

In [18]:
books['original_publication_year'] = books['original_publication_year'].fillna(0).astype('int')
books['original_publication_year'].describe()

count    10137.000000
mean      1951.095492
std        288.437688
min      -1750.000000
25%       1989.000000
50%       2004.000000
75%       2011.000000
max       2017.000000
Name: original_publication_year, dtype: float64

In [19]:
books['language_code'] = books['language_code'].apply(lambda x : 'eng' if x in  ['eng','en-US','en-GB','en-CA'] else 'other')
books['language_code'].value_counts()

eng      8807
other    1330
Name: language_code, dtype: int64

In [20]:
books.loc[books['original_publication_year'] <0,'original_publication_year'] =0
def publication_year_categorize(original_publication_year):
    if original_publication_year >=2015 :
        return('2015-2020')
    elif original_publication_year >= 2010:
        return('2010-2014')
    elif original_publication_year >= 2000:
        return('2000-2013')
    elif original_publication_year >= 1990:
        return('1990-2000')
    elif original_publication_year >= 1950:
        return('1950-1990')
    elif original_publication_year >= 1:
        return('<1950')
    else:
         return('unknown')
books['publication_timeframe'] =books['original_publication_year'].map(publication_year_categorize)
books['publication_timeframe'].value_counts()

2000-2013    3121
2010-2014    2552
1950-1990    1586
1990-2000    1360
<1950         814
2015-2020     515
unknown       189
Name: publication_timeframe, dtype: int64

###### Are the books unique ?

we have books unique isbn numbers  but  they have similar titles.
I am little wary of this data set now. It might  be a good  for proof of concept but  we may need to get cleaner data

In [22]:
#De-duplicating the  records as we only care about title and content of the book , 
books_with_same_title = (books['title'].value_counts()  )
print(f"duplicate book count = {len(books_with_same_title.index[books_with_same_title >1])}")

dup_books =books_with_same_title.index[books_with_same_title >1]
dup_book_df = books[books['title'].isin(dup_books)]

books = books.groupby('title').agg(
                             {
                              'book_id':'first',
                              'goodreads_book_id':'first',
                              'authors':'first', 
                              'description' :'first',
                              'original_publication_year': 'first',
                              'language_code' :'first',
                              'average_rating':'mean',
                              'ratings_count': 'sum',                              
                              'num_pages':'first',
                              'is_ebook':'first',
                              'ratings_count':'first'
                              }).reset_index()

duplicate book count = 57


In [25]:
book_tags = pd.read_csv("../data/book_tags_combined.csv")
###### Consolidate all Audio tags under same category
def consolidate_tags(tag_lst,target_tag):
    target_tag_id = book_tags[book_tags['tag_name']==target_tag]['tag_id'].values[0]  
    l= book_tags[(book_tags['tag_name'].isin(tag_lst))]['tag_name'].to_list()
    print(f"replacing {len(l)} tags with {target_tag}, tag_id= {target_tag_id}")
    book_tags['tag_name']= book_tags['tag_name'].apply(lambda x : target_tag if x in tag_lst else x)
    book_tags.loc[book_tags['tag_name'] ==target_tag,'tag_id']= target_tag_id 

In [ ]:
##### Consolidate all childrens books under same category
tag_lst= book_tags[(book_tags['tag_name'].str.contains('children')) ]['tag_name'].to_list()
consolidate_tags(list(set(tag_lst)),'childrens')

tag_lst= ['audio','audiobooks','audiobook','audible','audio-book','audio-books'] +(book_tags[(book_tags['tag_name'].str.contains('audiobook'))]['tag_name'].to_list())
consolidate_tags(set(tag_lst),'audiobook')

tag_lst = book_tags[(book_tags['tag_name'].str.contains('history'))]['tag_name'].to_list()
consolidate_tags(set(['war'] +tag_lst),'history')

#suspense
tag_lst= book_tags[(book_tags['tag_name'].str.contains('mystery'))]['tag_name'].to_list()
consolidate_tags(tag_lst,'mystery-thriller')


tag_lst=set(book_tags[(book_tags['tag_name'].str.contains('novel'))]['tag_name'].to_list())
consolidate_tags(tag_lst,'novels')

tag_lst=set(book_tags[(book_tags['tag_name'].str.contains('memoir'))]['tag_name'].to_list())
consolidate_tags(tag_lst,'biography')

## Fiction
consolidate_tags(['teen','juvenile','youth','new-adult'],'young-adult')
consolidate_tags(['general-fiction','speculative-fiction','magic','literary-fiction','contemporary-fiction','modern-fiction','vampires','vampire','fiction-to-read'],'fiction')
consolidate_tags(['ya-fiction','harry-potter'],'fiction') 
consolidate_tags(['sci-fi','scifi'],'science-fiction')
consolidate_tags(['teen-fiction','juvenile-fiction'],'young-adult-fiction')


consolidate_tags(['paranormal','paranormal-romance','paranormal-fantasy','fantasy-paranormal'],'supernatural') 

consolidate_tags(['fantasy-sci-fi,sci-fi-fantasy','science-fiction-fantasy','fantasy-scifi','sci-fi-and-fantasy'],'scifi-fantasy') 
consolidate_tags(['urban-fantasy','ya-fantasy','high-fantasy','sf-fantasy','guilty-pleasures'],'fantasy')

consolidate_tags(['funny','humour','comedy','humorous'],'humor') 
consolidate_tags(['20th-century','19th-century','historicals'],'historical') 
 
consolidate_tags(['non-fiction','to-read-nonfiction','non-fic','nonfiction-to-read'],'nonfiction') 
consolidate_tags(['to-read-non-fiction','non-fiction-to-read'],'nonfiction') 


consolidate_tags(['thrillers'],'thriller')
consolidate_tags(['childhood-reads','kids','kids-books','childhood-books','childhood','childhood-favorites'],'childrens')


consolidate_tags(['contemporary-romance'],'romance')
consolidate_tags(['urban-fantasy','ya-fantasy'],'fantasy')





consolidate_tags(['classic','literature','literary','classics','american-lit','american-literature','british-literature','chic-lit','kid-lit','chicklit','to-read-classics'],'classic-literature') 
consolidate_tags(['dystopia'],'dystopian')



consolidate_tags(['realistic'],'realistic-fiction')
consolidate_tags(['erotica','erotic'],'adult-romance')
consolidate_tags(['modern'],'contemporary')
consolidate_tags(['suspense-thriller','thriller-mystery','thriller-suspense','mystery-thrillers','epic-fantasy','mystery-suspense-thriller','mystery-thriller-suspense'],'mystery-thriller')


replacing 14759 tags with childrens, tag_id= 6953
replacing 33017 tags with audiobook, tag_id= 3389
replacing 6577 tags with history, tag_id= 14552
replacing 15571 tags with mystery-thriller, tag_id= 20994
replacing 17903 tags with novels, tag_id= 22034
replacing 4407 tags with young-adult, tag_id= 33114
replacing 13064 tags with fiction, tag_id= 11743
replacing 4066 tags with supernatural, tag_id= 29011
replacing 3069 tags with scifi-fantasy, tag_id= 26897
replacing 4483 tags with humor, tag_id= 15048
replacing 2507 tags with historical, tag_id= 14467
replacing 3396 tags with science-fiction, tag_id= 26837
replacing 3857 tags with nonfiction, tag_id= 21773
replacing 1593 tags with fiction, tag_id= 11743
replacing 4542 tags with fantasy, tag_id= 11305
replacing 1353 tags with thriller, tag_id= 30358
replacing 6100 tags with childrens, tag_id= 6953
replacing 1224 tags with audiobook, tag_id= 3389
replacing 1748 tags with nonfiction, tag_id= 21773
replacing 1105 tags with romance, tag_id

In [ ]:
books.to_csv("../data/books_desc.csv", index=False)

######  references
https://pypi.org/project/Goodreads/

https://betterreads.readthedocs.io/en/latest/

Kaggle data source : https://www.kaggle.com/zygmunt/goodbooks-10k

data set location : https://github.com/zygmuntz/goodbooks-10k

#https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html